In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:

plt.rcParams["figure.figsize"] = (10,6)

In [3]:
file_path = r"F:\Git learing\My-projects\Netflix-analysis\netflix_titles.csv"
df = pd.read_csv(file_path)

print("Dataset Loaded Successfully!")
print("Shape of dataset:", df.shape)
df.head()

Dataset Loaded Successfully!
Shape of dataset: (8807, 12)


,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,NaN,United States,9/25/2021,2020,PG-13,90 min,Documentaries,"As her father nears the end of his life, filmm..."
1,s2,TV Show,Blood & Water,NaN,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...",South Africa,9/24/2021,2021,TV-MA,2 Seasons,"International TV Shows, TV Dramas, TV Mysteries","After crossing paths at a party, a Cape Town t..."
2,s3,TV Show,Ganglands,Julien Leclercq,"Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...",NaN,9/24/2021,2021,TV-MA,1 Season,"Crime TV Shows, International TV Shows, TV Act...",To protect his family from a powerful drug lor...
3,s4,TV Show,Jailbirds New Orleans,NaN,NaN,NaN,9/24/2021,2021,TV-MA,1 Season,"Docuseries, Reality TV","Feuds, flirtations and toilet talk go down amo..."
4,s5,TV Show,Kota Factory,NaN,"Mayur More, Jitendra Kumar, Ranjan Raj, Alam K...",India,9/24/2021,2021,TV-MA,2 Seasons,"International TV Shows, Romantic TV Shows, TV ...",In a city of coaching centers known to train I...


In [4]:
print("\nDataset Info:")
print(df.info())

print("\nMissing Values:")
print(df.isnull().sum())


Dataset Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8807 entries, 0 to 8806
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   show_id       8807 non-null   object
 1   type          8807 non-null   object
 2   title         8807 non-null   object
 3   director      6173 non-null   object
 4   cast          7982 non-null   object
 5   country       7976 non-null   object
 6   date_added    8797 non-null   object
 7   release_year  8807 non-null   int64 
 8   rating        8803 non-null   object
 9   duration      8804 non-null   object
 10  listed_in     8807 non-null   object
 11  description   8807 non-null   object
dtypes: int64(1), object(11)
memory usage: 825.8+ KB
None

Missing Values:
show_id            0
type               0
title              0
director        2634
cast             825
country          831
date_added        10
release_year       0
rating             4
duration           3
liste

In [5]:
# Convert date column to datetime
df['date_added'] = pd.to_datetime(df['date_added'], errors='coerce')

# Create year and month columns
df['year_added'] = df['date_added'].dt.year
df['month_added'] = df['date_added'].dt.month

# Fill missing values
df['country'] = df['country'].fillna("Unknown")
df['rating'] = df['rating'].fillna("Not Rated")
df['director'] = df['director'].fillna("Unknown")
df['cast'] = df['cast'].fillna("Unknown")

print("Data Cleaning Completed!")


Data Cleaning Completed!


In [6]:
import mysql.connector
from mysql.connector import Error


In [7]:
try:
    connection = mysql.connector.connect(
        host='localhost',
        user='root',
        password='1234',
        database='netflix_db'   # Make sure this database exists
    )

    if connection.is_connected():
        print("Connected to MySQL successfully!")

except Error as e:
    print("Error while connecting to MySQL:", e)

Connected to MySQL successfully!


In [8]:
cursor = connection.cursor()

create_table_query = """
CREATE TABLE IF NOT EXISTS netflix_titles (
    show_id VARCHAR(20),
    type VARCHAR(20),
    title VARCHAR(255),
    director TEXT,
    cast TEXT,
    country VARCHAR(255),
    date_added DATE,
    release_year INT,
    rating VARCHAR(20),
    duration VARCHAR(50),
    listed_in TEXT,
    description TEXT,
    year_added INT,
    month_added INT
);
"""

cursor.execute(create_table_query)
connection.commit()

print("Table created successfully!")

Table created successfully!


In [9]:
# Replace NaN with None properly
df = df.replace({np.nan: None})

insert_query = """
INSERT INTO netflix_titles (
    show_id, type, title, director, cast, country,
    date_added, release_year, rating, duration,
    listed_in, description, year_added, month_added
) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
"""

data = df.values.tolist()

try:
    batch_size = 1000
    for i in range(0, len(data), batch_size):
        batch = data[i:i+batch_size]
        cursor.executemany(insert_query, batch)
        connection.commit()
        print(f"Inserted rows {i} to {i+batch_size}")

    print("All data inserted successfully!")

except Error as e:
    print("Error while inserting data:", e)

finally:
    cursor.close()
    connection.close()
    print("MySQL connection closed.")


Inserted rows 0 to 1000
Inserted rows 1000 to 2000
Inserted rows 2000 to 3000
Inserted rows 3000 to 4000
Inserted rows 4000 to 5000
Inserted rows 5000 to 6000
Inserted rows 6000 to 7000
Inserted rows 7000 to 8000
Inserted rows 8000 to 9000
All data inserted successfully!
MySQL connection closed.
